In [1]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

table4_1 = pd.read_excel('Table 4_1.xlsx')
table4_4 = pd.read_excel('Table 4_4.xlsx')
table5_1 = pd.read_excel('Table 5_1.xlsx')
table20 = pd.read_excel('Table 20.xlsx')
table19 = pd.read_excel('Table 19.xlsx')
clause26_2_1_1 = pd.read_excel('clause 26_2_1_1.xlsx')

In [2]:
# inputs to be taken
beam_span = float(input("Enter beam span in  meter: "))
overall_depth_beam = float(input("Enter depth of beam(mm): "))
width_beam = float(input("Enter the width of beam(mm): "))
live_load = float(input("Enter live load in kN/m: "))
imposed_load = float(input("Enter imposed load in kN/m: "))
floor_finish = float(input("Enter floor finish in kN/m: "))
fy = int(input("Enter the value of steel grade(250,415,500): "))
fck = int(input("Enter the value of concrete grade(20, 25, 30, 35, 40): "))

Enter beam span in  meter: 8
Enter depth of beam(mm): 650
Enter the width of beam(mm): 300
Enter live load in kN/m: 30
Enter imposed load in kN/m: 0
Enter floor finish in kN/m: 0
Enter the value of steel grade(250,415,500): 415
Enter the value of concrete grade(20, 25, 30, 35, 40): 20


In [10]:
self_weight_beam = 25* (overall_depth_beam/1000) * (width_beam/1000)
total_load = live_load + imposed_load + floor_finish + self_weight_beam
factored_load = 1.5* total_load
design_moment = (factored_load * (beam_span**2))/8
design_shear_force = (factored_load * beam_span)/2
effective_cover = table4_4.iloc[0,int(fck/5)-3]
effective_depth_beam = overall_depth_beam - effective_cover
if (fy==250):
    x_u = 0.53 * (effective_depth_beam/1000)
elif (fy==415):
    x_u = 0.48 * (effective_depth_beam/1000)
else:
    x_u = 0.46 * (effective_depth_beam/1000)


limiting_moment_resistance = 0.36 * fck * (width_beam) * x_u * ((effective_depth_beam/1000) - (x_u *0.42))


print(f"limiting moment: {limiting_moment_resistance}, design _moment: {design_moment}")
if (limiting_moment_resistance < design_moment):
    print("Doubly reinforcement beam is required")
    index_temp_y = int(np.floor(((effective_cover/ effective_depth_beam)*100)/5))
    if ((effective_cover/ effective_depth_beam)<0.05):
        fsc =min((( 0.0035 * (x_u * 1000 - effective_cover)*200000)/(x_u * 1000)), 0.87*fy)
    else:
        if (fy == 250):
            fsc = ((table5_1.iloc[1,index_temp_y] + (((effective_cover/ effective_depth_beam)-table5_1.iloc[0,index_temp_y])*(0.5/(table5_1.iloc[1,index_temp_y +1] - table5_1.iloc[1,index_temp_y])))),0.87*fy)

        elif (fy == 415):
            fsc = ((table5_1.iloc[2,index_temp_y] + (((effective_cover/ effective_depth_beam)-table5_1.iloc[0,index_temp_y])*(0.5/(table5_1.iloc[2,index_temp_y +1] - table5_1.iloc[2,index_temp_y])))),0.87*fy)

        else:
            fsc = ((table5_1.iloc[3,index_temp_y] + (((effective_cover/ effective_depth_beam)-table5_1.iloc[0,index_temp_y])*(0.5/(table5_1.iloc[3,index_temp_y +1] - table5_1.iloc[3,index_temp_y])))),0.87*fy)
    
    
    
    
    Asc = ((design_moment - limiting_moment_resistance)/(fsc * (effective_depth_beam - effective_cover)))*(10**6)
    # bars required to be taken
    if (Asc < 100):
        
        diameter_rod_compression = 8
        number_bars_compression = 2
        print(f"you need {number_bars_compression} compresssion bars of diameter {diameter_rod_compression}")
        
    else:
        bars_diameter = np.array([8,10,12,14,16,18,20,22,25,28,32,40])
            
        for i in range(len(bars_diameter)):
            number_bars_available = np.ceil((Asc/((np.pi)* (1/4) * (bars_diameter[len(bars_diameter)-i-1] **2))))
            
            if (number_bars_available >=2 and ((width_beam - (number_bars_available * bars_diameter[len(bars_diameter)-i-1]))/(number_bars_available-1)) >= 25 ):
                break
            else:
                continue
            i += 1
        diameter_rod_compression = int(input(f"Enter the the diameter of bar you want to take in mm{bars_diameter[:len(bars_diameter)-i-1]}: "))
        number_bars_compression = np.ceil((Asc/((np.pi)* (1/4) * (diameter_rod_compression **2))))
        print(f"you need {number_bars_compression} compresssion bars of diameter {diameter_rod_compression}")
    
    
    Ast = (design_moment/(0.87*fy*(effective_depth_beam - 0.42*(x_u*1000))))*(10**6)
    
    #bars details required
    if (Ast < 226):
        diameter_rod_tension = 12
        number_bars_tension = 2
        print(f"you need {number_bars_tension} tension bars of diameter {diameter_rod_tension}")
    else:
        bars_diameter = np.array([12,14,16,18,20,22,25,28,32,40])
            
        for i in range(len(bars_diameter)):
            number_bars_available = np.ceil((Ast/((np.pi)* (1/4) * (bars_diameter[len(bars_diameter)-i-1] **2))))
            
            if (number_bars_available >=2 and ((width_beam - (number_bars_available * bars_diameter[len(bars_diameter)-i-1]))/(number_bars_available-1)) >= 25 ):
                break
            else:
                continue
            i += 1
        diameter_rod_tension = int(input(f"Enter the the diameter of bar you want to take in mm{bars_diameter[:len(bars_diameter)-i-1]}: "))
        number_bars_tension = np.ceil((Ast/((np.pi)* (1/4) * (diameter_rod_tension **2))))
        print(f"you need {number_bars_tension} tension bars of diameter {diameter_rod_tension}")
    
    
    #location of bent up bar
    i = 1
    position=np.array([])
    while(i * (np.sqrt(2)*(effective_depth_beam - (x_u*1000))) < (beam_span*1000)/4):
        
        position = np.append(position, i * (np.sqrt(2)*(effective_depth_beam - (x_u*1000))))
        i += 1
    parts_beam =i
    factored_shear_force = (factored_load * beam_span)/2
    
    nominal_shear_stress = (factored_shear_force / (width_beam*effective_depth_beam))*(10**3)
    max_shear_stress = table20.iloc[0,int((fck/5)-3)]
    if (nominal_shear_stress > max_shear_stress):
        print(f"M{fck} concrete can hold that please change concrete grade and calculations below are useless !!!\n\n\n")
    else :

        #for bars at position greater than 2
        percentage_reinforcement_support = ((number_bars_tension ) * (np.pi/4) * (diameter_rod_tension **2) *100)/(effective_depth_beam * width_beam)
        if (( fck - 10)/5 <= 6):
            concrete_index_y = int(( fck - 10)/5)
        else:
            concrete_index_y = 6
        if (percentage_reinforcement_support<= 0.15):
            concrete_index_x = 1
            torque_c = .28
        elif (percentage_reinforcement_support >= 3):
            concrete_index_x = 13
            torque_c = .71
        else:
            concrete_index_x = int((np.floor(percentage_reinforcement_support*100))//25) + (1)
            torque_c = (((percentage_reinforcement_support - table19.iloc[concrete_index_x,0])*(table19.iloc[concrete_index_x,concrete_index_y]-table19.iloc[concrete_index_x+ 1,concrete_index_y ]))/(table19.iloc[concrete_index_x,0]-table19.iloc[concrete_index_x+ 1,0 ])) + table19.iloc[concrete_index_x,concrete_index_y]
        v_cu = (torque_c* width_beam* effective_depth_beam)*(10**(-3))
        

        #looping the portions
        percentage_reinforcement_support = ((number_bars_tension - np.floor(number_bars_tension/3)) * (np.pi/4) * (diameter_rod_tension **2) *100)/(effective_depth_beam * width_beam)
        for i in range(0,parts_beam):
            if (i==0):
            
                if (( fck - 10)/5 <= 6):
                    concrete_index_y = int(( fck - 10)/5)
                else:
                    concrete_index_y = 6
                if (percentage_reinforcement_support<= 0.15):
                    concrete_index_x = 1
                    torque_c = .28
                elif (percentage_reinforcement_support >= 3):
                    concrete_index_x = 13
                    torque_c = .71
                else:
                    concrete_index_x = int((np.floor(percentage_reinforcement_support*100))//25) + (1)
                    torque_c = (((percentage_reinforcement_support - table19.iloc[concrete_index_x,0])*(table19.iloc[concrete_index_x,concrete_index_y]-table19.iloc[concrete_index_x+ 1,concrete_index_y ]))/(table19.iloc[concrete_index_x,0]-table19.iloc[concrete_index_x+ 1,0 ])) + table19.iloc[concrete_index_x,concrete_index_y]
                v_cu = (torque_c* width_beam* effective_depth_beam)*(10**(-3))
                v_u = factored_shear_force
                v_us = v_u - v_cu
                spacing = min((0.87* fy* 2*(np.pi/4)*(8**2) * effective_depth_beam)/(v_us/2), (0.87* fy* 2*(np.pi/4)*(8**2)/(0.4*width_beam)), min((00.75* effective_depth_beam), 300))
                print(f"for portion{ 1} provide 2 legged 8mm diameter bars at {spacing}mm")
                
            else:
                
                if (( fck - 10)/5 <= 6):
                    concrete_index_y = int(( fck - 10)/5)
                else:
                    concrete_index_y = 6
                if (percentage_reinforcement_support<= 0.15):
                    concrete_index_x = 1
                    torque_c = .28
                elif (percentage_reinforcement_support >= 3):
                    concrete_index_x = 13
                    torque_c = .71
                else:
                    concrete_index_x = int((np.floor(percentage_reinforcement_support*100))//25) + (1)
                    torque_c = (((percentage_reinforcement_support - table19.iloc[concrete_index_x,0])*(table19.iloc[concrete_index_x,concrete_index_y]-table19.iloc[concrete_index_x+ 1,concrete_index_y ]))/(table19.iloc[concrete_index_x,0]-table19.iloc[concrete_index_x+ 1,0 ])) + table19.iloc[concrete_index_x,concrete_index_y]
                v_cu = (torque_c* width_beam* effective_depth_beam)*(10**(-3))
                distance_v_cu = ((beam_span*1000)/2)*(v_cu / factored_shear_force)
                v_u= ((((beam_span*1000)/2) - position[0] - ((i-1) * ((((beam_span*1000)/2)- position[0])/parts_beam)) ) /((beam_span*1000)/2) ) * factored_shear_force
                v_us = v_u - v_cu 
                if (v_us > 0):
                    spacing = min((0.87* fy* 2*(np.pi/4)*(8**2) * effective_depth_beam)/(v_us), (0.87* fy* 2*(np.pi/4)*(8**2)/(0.4*width_beam)), min((00.75* effective_depth_beam), 300))
                    print(f"portion {i+1} starts at {(((beam_span*1000)/2) - position[0] - ((i-1) * ((((beam_span*1000)/2)- position[0])/parts_beam)) )} from mid of span and provide 2 legged 8mm diameter bars at {spacing}mm ")
                else:
                    print(f"portion {i+1} starts at {(((beam_span*1000)/2) - position[0] - ((i-1) * ((((beam_span*1000)/2)- position[0])/parts_beam)) )} from mid of span and provide 2 legged 8mm diameter bars at 250 mm ")

    #development length
    development_length_compression = (diameter_rod_compression * 0.87*fy)/(4*1.6*clause26_2_1_1.iloc[0,int ((fck/5)-3)])
    development_length_tension = (diameter_rod_tension * 0.87*fy)/(4*1.6*clause26_2_1_1.iloc[0,int((fck/5)-3)])
    print(f"Development length compression in mm {development_length_compression}")
    print(f"Devlopment length tension in mm {development_length_tension}")
    
    #deflection check
    fs = 0.58 * fy *(Ast/(number_bars_tension * ((np.pi)* (1/4) * (diameter_rod_tension **2))))
            
    if((beam_span) > 10):
        beta =  10/beam_span
    else:
        beta = 1
    
    gamma = min(1/(0.225 + (0.0032 *fs) + (0.625/2.303)*np.log(percentage_reinforcement_support)),2)
    percentage_variable = (100*Asc)/(width_beam * effective_depth_beam)
    lmda = min(((1.6 * percentage_reinforcement_support)/(percentage_reinforcement_support + 0.275)),1.5)
    
    if(((beam_span *1000)/overall_depth_beam) > (26*beta*gamma*lmda)):
        print("sahi data lo na baby ji")


else:
    print("Singly reinforcement beam is required")
    Asc = 0
    print("We take 2 8mm bars above as Asc =0 ")
    Ast = (0.362 * fck * width_beam * (x_u*1000))/(0.87*fy) 
    
    
    # bar details required
    if (Ast < 226):
        print("Take 2 12mm bars")
    else:
        bars_diameter = np.array([12,14,16,18,20,22,25,28,32,40])
            
        for i in range(len(bars_diameter)):
            number_bars_available = np.ceil((Ast/((np.pi)* (1/4) * (bars_diameter[len(bars_diameter)-i-1] **2))))
            
            if (number_bars_available >=2 and ((width_beam - (number_bars_available * bars_diameter[len(bars_diameter)-i-1]))/(number_bars_available-1)) >= 25 ):
                break
            else:
                continue
            i += 1
        diameter_rod_tension = int(input(f"Enter the the diameter of bar you want to take in mm{bars_diameter[:len(bars_diameter)-i-1]}: "))
        number_bars_tension = np.ceil((Ast/((np.pi)* (1/4) * (diameter_rod_tension **2))))
        print(f"you need {number_bars_tension} tension bars of diameter {diameter_rod_tension}")


limiting moment: 328.546326528, design _moment: 418.5
Doubly reinforcement beam is required
Enter the the diameter of bar you want to take in mm[ 8 10 12 14 16 18 20]: 12
you need 4.0 compresssion bars of diameter 12
Enter the the diameter of bar you want to take in mm[12 14 16 18 20 22 25 28 32]: 16
you need 12.0 tension bars of diameter 16
for portion1 provide 2 legged 8mm diameter bars at 300mm
portion 2 starts at 3536.703636966574 from mid of span and provide 2 legged 8mm diameter bars at 300mm 
portion 3 starts at 2829.362909573259 from mid of span and provide 2 legged 8mm diameter bars at 300mm 
portion 4 starts at 2122.022182179944 from mid of span and provide 2 legged 8mm diameter bars at 250 mm 
portion 5 starts at 1414.6814547866297 from mid of span and provide 2 legged 8mm diameter bars at 250 mm 
Development length compression in mm 564.140625
Devlopment length tension in mm 752.1875
